In [1]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
import matplotlib.pyplot as plt

from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

from nixtla.models.esrnn.esrnn import ESRNN

In [2]:
from nixtla.data.datasets.epf import EPF, EPFInfo
Y_df, X_df = EPF.load(directory='data', group=EPFInfo.groups[0])
train_outsample_mask = np.ones(len(Y_df))
train_outsample_mask[-365 * 24:] = 0
sum(train_outsample_mask)
epf_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=X_df, ts_train_mask=train_outsample_mask)
Y_df.head()

Processing dataframes ...
Creating ts tensor ...


,unique_id,ds,y
0,NP,2013-01-01 00:00:00,31.05
1,NP,2013-01-01 01:00:00,30.47
2,NP,2013-01-01 02:00:00,28.92
3,NP,2013-01-01 03:00:00,27.88
4,NP,2013-01-01 04:00:00,26.96


In [3]:
train_loader = TimeSeriesLoaderGeneral(ts_dataset=epf_dataset,
                                    model='esrnn',
                                    offset=24,
                                    window_sampling_limit= 90*1*24, #365*4*24
                                    input_size=7*24,
                                    output_size=24,
                                    idx_to_sample_freq=1,
                                    batch_size=1,
                                    is_train_loader=True)

In [4]:
esrnn = ESRNN(input_size=7*24,
              output_size=24,
              max_epochs=5,
              freq_of_test=-1,
              learning_rate=1e-3,
              lr_scheduler_step_size=9,
              lr_decay=0.9,
              per_series_lr_multip=1.0,
              gradient_eps=1e-8,
              gradient_clipping_threshold=20,
              rnn_weight_decay=0,
              noise_std=0.001,
              level_variability_penalty=200,
              testing_percentile=50,
              training_percentile=50,
              cell_type='LSTM',
              state_hsize=40,
              dilations=[[1, 2], [4, 8]],
              add_nl_layer=False,
              seasonality=[24],
              random_seed=1,
              device='cpu')

In [ ]:
esrnn.fit(train_ts_loader=train_loader, eval_epochs=1)

In [ ]:
y_hat = esrnn.predict(ts_loader=train_loader)

In [ ]:
y_plot = Y_df['y'][-168:]
y_hat_plot = y_hat['y_hat']
plt.plot(range(len(y_plot)), y_plot)
plt.plot(range(len(y_plot), len(y_plot)+len(y_hat_plot)), y_hat_plot)

# TOURISM WITH RANDOM STATIC

In [ ]:
from nixtla.data.datasets.tourism import Tourism, TourismInfo
group = TourismInfo.groups[0]
print("TourismInfo.groups[0]", group)
Y_df, _ = Tourism.load(directory='data', group=group)

S_df = Y_df.copy()
S_df = S_df.drop_duplicates(subset='unique_id').reset_index(drop=True)
S_df = S_df.drop(columns=['ds','y'])
S_df['static1'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static2'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static3'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static4'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static5'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static6'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static7'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static8'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static9'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static10'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static11'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static12'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static13'] = np.random.randint(low=0, high=10, size=len(S_df))
S_df['static14'] = np.random.randint(low=0, high=10, size=len(S_df))

In [ ]:
tourism_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=S_df, X_df=None, ts_train_mask=None)

In [ ]:
train_loader = TimeSeriesLoaderGeneral(ts_dataset=tourism_dataset,
                                            model='esrnn',
                                            offset=4,
                                            window_sampling_limit=20*4, 
                                            input_size=1*4,
                                            output_size=4,
                                            idx_to_sample_freq=1,
                                            batch_size= 32,
                                            n_series_per_batch=32,
                                            is_train_loader=True)

In [ ]:
esrnn = ESRNN(input_size=1*4,
              output_size=4,
              max_epochs=5,
              freq_of_test=-1,
              learning_rate=1e-3,
              lr_scheduler_step_size=9,
              lr_decay=0.9,
              per_series_lr_multip=1.0,
              gradient_eps=1e-8,
              gradient_clipping_threshold=20,
              rnn_weight_decay=0,
              noise_std=0.001,
              level_variability_penalty=200,
              testing_percentile=50,
              training_percentile=50,
              cell_type='LSTM',
              state_hsize=40,
              dilations=[[1, 2], [4, 8]],
              add_nl_layer=False,
              seasonality=[],
              random_seed=1,
              device='cpu')

In [ ]:
esrnn.fit(train_ts_loader=train_loader, eval_epochs=1)

In [ ]:
y_hat = esrnn.predict(ts_loader=train_loader)

In [ ]:
y_plot = Y_df[Y_df['unique_id']=='Y1']['y']
y_hat_plot = y_hat[y_hat['unique_id']=='Y1']['y_hat']
plt.plot(range(len(y_plot)), y_plot)
plt.plot(range(len(y_plot), len(y_plot)+len(y_hat_plot)), y_hat_plot)

In [ ]:
y_test = Y_df.groupby('unique_id').tail(4).reset_index(drop=True)
np.mean(np.abs(y_test['y']-y_hat['y_hat'])/np.abs(y_test['y']))